<a href="https://colab.research.google.com/github/jyoshika1976/CodeSoft/blob/main/recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas scikit-learn


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Sample dataset of users and their ratings for various movies
data = {
    'User': ['User1', 'User1', 'User1', 'User2', 'User2', 'User3', 'User3', 'User4', 'User4', 'User4'],
    'Movie': ['Inception', 'Titanic', 'Avengers', 'Inception', 'Avatar', 'Titanic', 'Avatar', 'Inception', 'Titanic', 'Avengers'],
    'Rating': [5, 4, 3, 4, 5, 3, 4, 2, 5, 4]
}

# Convert the data into a pandas DataFrame
df = pd.DataFrame(data)

# Create a pivot table with users as rows and movies as columns
user_movie_matrix = df.pivot_table(index='User', columns='Movie', values='Rating')

# Fill missing values with 0 (since the user hasn't rated the movie)
user_movie_matrix.fillna(0, inplace=True)

print("User-Movie Matrix:\n", user_movie_matrix)

# Compute the similarity between users using cosine similarity
user_similarity = cosine_similarity(user_movie_matrix)

# Create a DataFrame to display similarity between users
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

print("\nUser Similarity Matrix:\n", user_similarity_df)

def recommend_movies(user, user_movie_matrix, user_similarity_df):
    """
    Recommend movies for a user based on ratings from similar users.
    """
    # Find the most similar users to the given user
    similar_users = user_similarity_df[user].sort_values(ascending=False).drop(user)

    # Get the user's current ratings
    user_ratings = user_movie_matrix.loc[user]

    # Movies the user hasn't rated yet
    unrated_movies = user_ratings[user_ratings == 0].index

    # Dictionary to store movie recommendations with weighted ratings
    recommendations = {}

    # Loop through similar users
    for similar_user, similarity in similar_users.items():# Changed line
        # Get the similar user's ratings
        similar_user_ratings = user_movie_matrix.loc[similar_user]

        # Get movies that the similar user has rated and the target user hasn't
        for movie in unrated_movies:
            if similar_user_ratings[movie] > 0:
                if movie not in recommendations:
                    recommendations[movie] = 0
                # Weighted score based on similarity
                recommendations[movie] += similar_user_ratings[movie] * similarity

    # Sort recommendations by weighted score
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)

    return sorted_recommendations

# Recommend movies for User1
recommendations = recommend_movies('User1', user_movie_matrix, user_similarity_df)

print("\nRecommended Movies for User1:")
for movie, score in recommendations:
    print(f"{movie}: {score:.2f}")


User-Movie Matrix:
 Movie  Avatar  Avengers  Inception  Titanic
User                                       
User1     0.0       3.0        5.0      4.0
User2     5.0       0.0        4.0      0.0
User3     4.0       0.0        0.0      3.0
User4     0.0       4.0        2.0      5.0

User Similarity Matrix:
 User      User1     User2     User3     User4
User                                         
User1  1.000000  0.441726  0.339411  0.885438
User2  0.441726  1.000000  0.624695  0.186248
User3  0.339411  0.624695  1.000000  0.447214
User4  0.885438  0.186248  0.447214  1.000000

Recommended Movies for User1:
Avatar: 3.57
